In [2]:
# import statements 
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense, Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical 



2024-03-13 17:36:57.677982: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-13 17:36:58.991815: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-13 17:37:02.268794: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 17:37:06.097043: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# defining our sample names/data 
character_names = ['gabriella', 'ryan', 'cecilia', 'kari', 'dimmer', 'allan', 'sarah', 'eliza', 'aaron', 'katnis', 'henry', 'ava', 'alice', 'bob', 'jon', 'alberto', 'marcos', 'vex', 
                   'zach', 'shahbaaz', 'fourohfour', 'pharoah', 'mia', 'katherine', 'farrow', 'ivy', 'jack', 'emily', 'bob', 'charlie', 'radhika', 
                   'quinn', 'patrick', 'eva', 'diana', 'jeffrey', 'noah', 'liam', 'puck', 'burt', 'juan', 'jose', 'chris']


# create a vocabulary based on the sample names
vocab = sorted(set(''.join(character_names)))
char_to_idx = {char: idx for idx, char in enumerate(vocab)}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

# convert character names to sequences 
sequences = [[char_to_idx[char] for char in name] for name in character_names]

# padding our sequences to ensure equal name length later 
max_name_length = max(len(seq) for seq in sequences)
print(type(max_name_length))
padded_sequences = pad_sequences(sequences, maxlen=max_name_length, padding='post')

# create input + target sequences 
X = padded_sequences[:, :-1]
y = padded_sequences[:, 1:]

# check the shape of our input sequence 
print(X.shape)

#check the shape of our target sequence 
print(y.shape)

# convert our target sequence into a one-hot encoding 
y_one_hot = to_categorical(y, num_classes=len(vocab))

#defining characteristics for our gru-based character generation model 
vocab_size = len(vocab)
embedding_dim = 32

# created the model
# running into issues with input_length = max_name_length-1 in our Embedding function inside the model
# return statement said it did not recognize the variable/wasn't a valid integer, but when printing the value type it returned int

model = tf.keras.Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim), 
    GRU(128, return_sequences=True), 
    Dense(vocab_size, activation='softmax')
])

# compile the model 
model.compile(optimizer='adam', loss = 'categorical_crossentropy')

# train the model 
model.fit(X, y_one_hot, epochs = 50, verbose=1)




<class 'int'>
(43, 9)
(43, 9)
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 3.2654
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.2143  
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.1572 
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.0862 
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.9960 
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.8460 
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.6702 
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.4001 
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.0649 
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.7636 
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.6806 
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.7667 
Epoch 13/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.7607 
Epoch 14/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.7828 
Epoch 15/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.7138 
Epo

In [11]:
# declare seed character 
seed = 'ch'

# map our seed characters to indexes 
seed_idx = [char_to_idx[char] for char in seed]

# actually calling our model 
# finding the probabilities from the model 
# generating a name ! 
for _ in range(3):
    temperature = .95 # higher the temp -> the more random output will be 

    predicted_probs = model.predict(np.array(seed_idx).reshape(1, -1))[0]
    predicted_probs = predicted_probs[-1] # take last predicted probability from model 

    predicted_probs = np.power(predicted_probs, 1 /temperature)
    predicted_probs = predicted_probs / np.sum(predicted_probs)

    next_char = np.random.choice(list(char_to_idx.keys()), p = predicted_probs)

    seed += next_char 
    seed_idx.append(char_to_idx[next_char])

print(seed)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
chija
